# Libraries and usefull functions

In [1]:
# Importing libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import json
import datetime

import pprint

# Test code

In [ ]:
# Opening web browser
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(
    ChromeDriverManager().install()), options=options)

# Getting the the page we want to scrap
driver.get('https://www.grainger.com/category/lighting/light-bulbs-lamps/circular-light-bulbs-lamps?categoryIndex=8')
driver.implicitly_wait(1)

# Extracting page main title
title_page = driver.find_element(By.CLASS_NAME, 'KkejIK').text
# Identification of each sections of the page. Creating an empty dictionary to save the data.
sections = driver.find_elements(By.CLASS_NAME, 'MAcbb-')
section_dict = {}

# Looping throught each section to get the information we want which are the tables
for section in sections:
    # Extracting section title
    title_section = section.find_element(By.CLASS_NAME, "sC0Aof").text
    sub_section_dict = {}
    section_dict[title_section] = sub_section_dict

    # if there is sub section, we looping throught ever sub-section to extract every table
    if section.find_elements(By.CLASS_NAME, "T8G8vu"):
        sub_sections = section.find_elements(By.CLASS_NAME, "T8G8vu")
        extra_title = ''
        for sub_section in sub_sections:

            try:
                title_sub_section = extra_title + \
                    sub_section.find_element(By.CLASS_NAME, 'SQoGqa').text
                sub_section_dict[title_sub_section] = {}

                # To load the HTML, we need to 'activate' the JS by clicking on a button of the table
                sub_section.find_element(By.CLASS_NAME, 'JxT10f').click()

                # Getting features name as they are different on each urls. Features comes in one long str thet needs to be split
                features = sub_section.find_element(By.CLASS_NAME, 'JxT10f')
                feature_list = (features.text).split('\n')

                # Getting the table associated with the sub section
                tables = sub_section.find_element(By.CLASS_NAME, 'cjpIYY')
                feature_dict = {}

                # Adding the date of scrapping
                current_date = datetime.datetime.now().strftime("%Y-%m-%d")

                # Going through every row of every column and creating the list of values associated with each feature
                rows = tables.find_elements(By.TAG_NAME, 'tr')
                for i in range(len(feature_list)):
                    feature_dict[feature_list[i]] = []
                    feature_dict['Date'] = []

                    for row in rows:
                        value = row.find_element(By.XPATH, f'./td[{i+1}]').text
                        feature_dict[feature_list[i]].append(value)
                        feature_dict['Date'].append(current_date)

                sub_section_dict[title_sub_section] = feature_dict

            except Exception as e:
                extra_title = sub_section.find_element(
                    By.CLASS_NAME, 'SQoGqa').text + ' '
                del sub_section_dict[title_sub_section]

    else:

        sub_section.find_element(By.CLASS_NAME, 'JxT10f').click()

        # Getting features name as they are different on each urls. Features comes in one long str thet needs to be split
        features = sub_section.find_element(By.CLASS_NAME, 'JxT10f')
        feature_list = (features.text).split('\n')

        # Getting the table associated with the sub section
        tables = sub_section.find_element(By.CLASS_NAME, 'cjpIYY')
        feature_dict = {}

        # Adding the date of scrapping
        current_date = datetime.datetime.now().strftime("%Y-%m-%d")

        # Going through every row of every column and creating the list of values associated with each feature
        rows = tables.find_elements(By.TAG_NAME, 'tr')
        for i in range(len(feature_list)):
            feature_dict[feature_list[i]] = []
            feature_dict['Date'] = []

            for row in rows:
                value = row.find_element(By.XPATH, f'./td[{i+1}]').text
                feature_dict[feature_list[i]].append(value)
                feature_dict['Date'].append(current_date)

        sub_section_dict[title_sub_section] = feature_dict


page_dict = {}
page_dict[title_page] = section_dict

pprint.pprint(page_dict)

driver.quit()

# Scraping the page

In [ ]:
def url_scraper(url):

    # Opening web browser
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(
        ChromeDriverManager().install()), options=options)

    # Getting the the page we want to scrap
    driver.get(url)
    driver.implicitly_wait(1)

    # Extracting page main title
    title_page = driver.find_element(By.CLASS_NAME, 'KkejIK').text
    # Identification of each sections of the page. Creating an empty dictionary to save the data.
    sections = driver.find_elements(By.CLASS_NAME, 'MAcbb-')
    section_dict = {}

    # Looping throught each section to get the information we want which are the tables
    for section in sections:
        # Extracting section title
        title_section = section.find_element(By.CLASS_NAME, "sC0Aof").text
        sub_section_dict = {}
        section_dict[title_section] = sub_section_dict

        # Looping throught ever sub-section to extract every table
        if section.find_elements(By.CLASS_NAME, "T8G8vu"):
            sub_sections = section.find_elements(By.CLASS_NAME, "T8G8vu")
            extra_title = ''
            for sub_section in sub_sections:

                try:
                    title_sub_section = extra_title + \
                        sub_section.find_element(By.CLASS_NAME, 'SQoGqa').text
                    sub_section_dict[title_sub_section] = {}

                    # To load the HTML, we need to 'activate' the JS by clicking on a button of the table
                    sub_section.find_element(By.CLASS_NAME, 'JxT10f').click()

                    # Getting features name as they are different on each urls. Features comes in one long str thet needs to be split
                    features = sub_section.find_element(
                        By.CLASS_NAME, 'JxT10f')
                    feature_list = (features.text).split('\n')

                    # Getting the table associated with the sub section
                    tables = sub_section.find_element(By.CLASS_NAME, 'cjpIYY')
                    feature_dict = {}

                    # Adding the date of scrapping
                    current_date = datetime.datetime.now().strftime("%Y-%m-%d")

                    # Going through every row of every column and creating the list of values associated with each feature
                    rows = tables.find_elements(By.TAG_NAME, 'tr')
                    for i in range(len(feature_list)):
                        feature_dict[feature_list[i]] = []
                        feature_dict['Date'] = []

                        for row in rows:
                            value = row.find_element(
                                By.XPATH, f'./td[{i+1}]').text
                            feature_dict[feature_list[i]].append(value)
                            feature_dict['Date'].append(current_date)

                    sub_section_dict[title_sub_section] = feature_dict

                except Exception as e:
                    extra_title = sub_section.find_element(
                        By.CLASS_NAME, 'SQoGqa').text + ' '
                    del sub_section_dict[title_sub_section]

        else:

            sub_section.find_element(By.CLASS_NAME, 'JxT10f').click()

            # Getting features name as they are different on each urls. Features comes in one long str thet needs to be split
            features = sub_section.find_element(By.CLASS_NAME, 'JxT10f')
            feature_list = (features.text).split('\n')

            # Getting the table associated with the sub section
            tables = sub_section.find_element(By.CLASS_NAME, 'cjpIYY')
            feature_dict = {}

            # Adding the date of scrapping
            current_date = datetime.datetime.now().strftime("%Y-%m-%d")

            # Going through every row of every column and creating the list of values associated with each feature
            rows = tables.find_elements(By.TAG_NAME, 'tr')
            for i in range(len(feature_list)):
                feature_dict[feature_list[i]] = []
                feature_dict['Date'] = []

                for row in rows:
                    value = row.find_element(By.XPATH, f'./td[{i+1}]').text
                    feature_dict[feature_list[i]].append(value)
                    feature_dict['Date'].append(current_date)

            sub_section_dict[title_sub_section] = feature_dict

    page_dict = {}
    page_dict[title_page] = section_dict

    driver.quit()

    return page_dict

# Scrapping follow up

In [ ]:
def urls_scrapped(urls):
    data = []
    # urls_done has been created if the script come to a stop for whatever reason. We can look at the last link added here and restart from here.
    urls_done = []
    for url in urls:
        print(url)  # Help to follow the script live
        try:
            scraping = url_scraper(url)
            data.append(scraping)
            urls_done.append(url)
        except:  # If there is an issue on a url we can just skip it and move on
            pass

    return data, urls_done[-1]

## Defining urls we want to scrap

In [ ]:
with open('urls_extractor/urls_extractor/spiders/urls.json', 'r') as file:
    datas = json.load(file)

urls_to_scrap = []

for data in datas:
    for key, value in data.items():
        urls_to_scrap.append(value)

u = urls_to_scrap[0:3]

In [ ]:
results = urls_scrapped(u)
scraping = results[0]
urls_processed = results[1]

# Saving to json

In [ ]:
def json__saving(page_dict):
    json_string = json.dumps(page_dict)
    current_date = datetime.datetime.now().strftime("%Y-%m-%d")
    with open(f'scrapings/{current_date}_mon_fichier.json', 'w') as datas:
        datas.write(json_string)

In [ ]:
json__saving(scraping)

# Here we should save scraping to json and save this json on the data base.